# lets see how bpe algorithm works and how encode and decode work 

In [1]:
# lets take an simple string of text
# reference : https://sidsite.com/posts/bpe/
text = "aa abc abc"

In [1]:
# first download the data that are going to used in this whole process
# get dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-12-03 15:13:22--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  2.99MB/s    in 0.4s    

2023-12-03 15:13:23 (2.99 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# define a dictnary to store our all token
tokens = {}

# add all the base character first and their count will be 1 as default
tokens = {i:1 for i in text if i not in tokens}
tokens

{'a': 1, ' ': 1, 'b': 1, 'c': 1}

In [12]:
# define a function that will return all pair of text having occurance in whole text
def get_pairs(text):
    pairs = {}
    next_token = None
    next_token_count = 0

    for i in range(len(text)-1):
        # skip (" ", <tok>) to avoid counting across words
        if text[i] == ' ':
            continue
        temp = text[i:i+2]
        if temp in pairs:
            pairs[temp] = pairs[temp] + 1
        else:
            pairs.update({temp:1})
        if pairs[temp] > next_token_count:
            next_token = temp
            next_token_count = pairs[temp]
    return pairs,(next_token,next_token_count)

pairs,next_token = get_pairs(text)
print(f'Next token : {next_token}')
print(f'Pairs of Token : {pairs}')

Next token : ('ab', 2)
Pairs of Token : {'aa': 1, 'a ': 1, 'ab': 2, 'bc': 2, 'c ': 1}


In [13]:
# after getting next token , add next_token to our tokens dictinary
tokens.update({next_token[0]: next_token[1]})
tokens

{'a': 1, ' ': 1, 'b': 1, 'c': 1, 'ab': 2}

In [19]:
# now repeate this
new_text = ' '.join(list(pairs.keys()))
print(new_text)
pairs,next_token = get_pairs(new_text)
print(f'Next token : {next_token}')
print(f'Pairs of Token : {pairs}')


aa a  ab b  bc c 
Next token : ('a ', 2)
Pairs of Token : {'aa': 1, 'a ': 2, 'ab': 1, 'b ': 2, 'bc': 1, 'c ': 2}


# huggingface tokenizer demo and know how to use encode and decode

In [2]:
path = 'hf_tokenizer.json'
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file(path)


In [8]:
# encode method 
text = '''hello my name is david 
i am from nepal'''
encode = tokenizer.encode(text)
print(f'encode output : {encode}')
print(f'encoded ids {encode.ids}')
print(f'encoded token {encode.tokens}')

encode output : Encoding(num_tokens=15, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
encoded ids [85, 81, 56, 99, 505, 75, 573, 63, 163, 50, 174, 244, 176, 57, 134]
encoded token ['he', 'll', 'o', 'my', 'name', 'is', 'da', 'v', 'id', 'i', 'am', 'from', 'ne', 'p', 'al']


In [12]:
# decode
decoded = tokenizer.decode(encode.ids)

In [13]:
decoded

'he ll o my name is da v id i am from ne p al'

In [15]:
# Let's tokenizer a simple input
from tokenizers.models import BPE
tokenizer.model = BPE.from_file('vocab.json', 'merges.txt')
encoding = tokenizer.encode("This is a simple input to be tokenized")

print("Encoded string: {}".format(encoding.tokens))

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded string: ['T', 'hi', 's', 'is', 'a', 's', 'i', 'mp', 'le', 'in', 'p', 'ut', 'to', 'b', 'e', 'to', 'k', 'en', 'iz', 'ed']
Decoded string: T hi s is a s i mp le in p ut to b e to k en iz ed


In [1]:
from datasets import load_dataset
dataset = load_dataset('imdb',split='train')

Generating unsupervised split: 100%|██████████| 50000/50000 [00:05<00:00, 8575.99 examples/s] 


In [4]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [5]:
dataset2 = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")


Generating train split: 100%|██████████| 20022/20022 [00:00<00:00, 945776.75 examples/s]


In [8]:
for item in dataset2:
    print(item)
    break

{'instruction': 'Create a function that takes a specific input and produces a specific output using any mathematical operators. Write corresponding code in Python.', 'input': '', 'output': 'def f(x):\n    """\n    Takes a specific input and produces a specific output using any mathematical operators\n    """\n    return x**2 + 3*x'}


In [9]:
# for chat dataset {'user','assistant'}

dataset3 = load_dataset("timdettmers/openassistant-guanaco", split="train")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")
Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 500.27it/s]
Generating train split: 9846 examples [00:00, 197081.83 examples/s]
Generating test split: 518 examples [00:00, 124821.87 examples/s]


In [11]:
for item in dataset3:
    print(item)
    break

{'text': '### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining po